# GEEO Tutorial 2 - Auxiliary exports

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/leonsnill/geeo/blob/master/docs/tutorial_2_auxiliary-exports.ipynb)

Geeo contains a variety of helper or auxiliary functions beyond the core processing pipeline. In this notebook we briefly illustrate how to export image data using geeo, including precisely aligning the export image to a given ROI (as learned in the previous tutorial).

### Exporting an ee.Image

In [ ]:
my_project_name =''

# imports
import ee
ee.Authenticate()
ee.Initialize(project=my_project_name)
try:
    import geeo
except:
    !pip install --quiet git+https://github.com/leonsnill/geeo.git
    import geeo

import geemap
import eerepr
eerepr.initialize()
import geopandas as gpd

# study area: create tiles -> select one tile (Berlin)
gdf_countries = gpd.read_file('https://raw.githubusercontent.com/leonsnill/geeo/master/geeo/data/ne_50m_admin_countries.gpkg')
germany = gdf_countries[gdf_countries['ADMIN'] == 'Germany']
germany_tiles = geeo.create_tiles(crs='EPSG:3035', vector_roi=germany, tile_size=30000, land_mask=True)
tile_berlin = 'X017-Y009'  # Berlin and norhthern Brandenburg
ROI = germany_tiles[germany_tiles['ID'] == tile_berlin]
ROI_FEATCOL = geeo.create_roi(ROI)['roi_featcol']  # convert to ee.FeatureCollection


In [ ]:
img_hansen = ee.Image("UMD/hansen/global_forest_change_2024_v1_12").select('treecover2000')

The study area:

In [ ]:
M = geemap.Map(center=[52.5, 14], zoom=9)
M.add_basemap('HYBRID')
vis_params = {'color': 'red', 'width': 2, 'lineType': 'solid','fillColor': '00000000'}
M.addLayer(img_hansen, {'bands': ['treecover2000'], 'palette': ['white', 'green'], 'min': 0, 'max': 100}, 'Hansen 2000 tree cover')
M.addLayer(ROI_FEATCOL.style(**vis_params), {}, 'Study Area')
M

To export the image precisely matching the ROI and using bilinear resampling simply call:

In [ ]:
'''
my_export = geeo.export_img(img=img_hansen,
                            px_res=30, region=ROI, crs='EPSG:3035', resampling_method='bilinear',
                            out_location='Asset', out_dir='projects/eexnill/assets/geeo_public', outname='global_forest_change_2024_v1_12_treecover2000')
'''

Visualizing the result:

In [ ]:
img_hansen = ee.Image('projects/eexnill/assets/geeo_public/global_forest_change_2024_v1_12_treecover2000')

M = geemap.Map(center=[52.316582, 13.661537], zoom=18)
M.add_basemap('HYBRID')
vis_params = {'color': 'red', 'width': 2, 'lineType': 'solid','fillColor': '00000000'}

M.addLayer(img_hansen, {'bands': ['treecover2000'], 'palette': ['white', 'green'], 'min': 0, 'max': 100}, 'Hansen 2000 tree cover')
M.addLayer(ROI_FEATCOL.style(**vis_params), {}, 'Study Area')
M

Note how the output image exactly aligns with our tiling scheme (possible since a 30m pixel fits into our 30x30km tiling scheme).

### Exporting an ee.ImageCollection
When working with ee.ImageCollections, we need to create an ee.Image prior to export. This either involves selecting a specific image in the collection or more frequently, creating a spatial mosaic.

#### Mosaicking
Consider we wanted to download the recently published satellite embeddings by Google DeepMind (https://developers.google.com/earth-engine/datasets/catalog/GOOGLE_SATELLITE_EMBEDDING_V1_ANNUAL) for a study area defined by a single tile of a gridding scheme we are using for Germany:

In [ ]:
# satellite embeddings for 2024
imgcol_embeddings = ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL').filterBounds(ROI_FEATCOL).filterDate('2024-01-01', '2024-12-31')
imgcol_embeddings.first()

We could simply mosaick the images to create a spatially continuous image. However, prior to that we might want to **set the resampling method**:

In [ ]:
imgcol_embeddings = imgcol_embeddings.map(lambda img: img.resample('bilinear'))
img_embeddings = imgcol_embeddings.mosaic()
img_embeddings

Now we simply call the export like before:

In [ ]:
'''
my_export = geeo.export_img(img=img_embeddings,
                            px_res=10, region=ROI, crs='EPSG:3035',
                            out_location='Asset', out_dir='projects/eexnill/assets/geeo_public', outname='GOOGLE_SATELLITE_EMBEDDING_V1_2024')
'''

In [ ]:
img_embeddings = ee.Image('projects/eexnill/assets/geeo_public/GOOGLE_SATELLITE_EMBEDDING_V1_2024')
img_embeddings

In [ ]:
M = geemap.Map(center=[52.46, 13.44], zoom=10)
M.add_basemap('HYBRID')
vis_params = {'color': 'red', 'width': 2, 'lineType': 'solid','fillColor': '00000000'}

M.addLayer(img_embeddings, {'bands': ['A00', 'A01', 'A03'], 'min': -0.34, 'max': 0}, 'Embeddings')
M.addLayer(ROI_FEATCOL.style(**vis_params), {}, 'Study Area')
M

If an ee.ImageCollection also has a temporal domain like the embedding dataset, and we want to export all images, we can use the `imgcol_to_img()` function from geeo. This will append the date information to the bandname:

In [ ]:
from geeo import imgcol_to_img
imgcol_embeddings = ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL').filterBounds(ROI_FEATCOL).filterDate('2023-01-01', '2024-12-31').sort('system:time_start')
img_embeddings_all = imgcol_to_img(imgcol_embeddings)
img_embeddings_all